In [1]:
%matplotlib inline
%load_ext autoreload
% autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import seaborn

import ising
import exact_ising_model as exact
import plotting
import analysis
import simulation

In [ ]:
ising = metropolis_ising.MetropolisIsing(lattice_size=4, bond_energy=1, temperature=2.2, initial_temperature="hi", sweeps=10000)
plotting.show_lattice(ising.lattice)

In [ ]:
ising.metropolis(False)

plotting.show_history(ising.magnetization_history, "Magnetization")
plotting.show_history(ising.energy_history, "Energy")

In [ ]:
# Data gets normalized immediatly.
equilibrium = 5000
equilibrium_energy = ising.energy_history[equilibrium:] / ising.no_of_sites
equilibrium_magnetization = ising.magnetization_history[equilibrium:] / ising.no_of_sites

In [ ]:
def binning_temperature_range(lattice_size, lower, upper, step=0.2):
    energy_trange = []
    energy_error_trange = []
    energy_correlation_trange = []
    
    magnetization_trange = []
    magnetization_error_trange = []
    magnetization_correlation_trange = []
    
    specific_heat_trange = []
    specific_heat_error_trange = []
    
    for temperature in np.arange(lower, upper, step):
        ising = metropolis_ising.MetropolisIsing(lattice_size=lattice_size,
                                                 bond_energy=1,
                                                 temperature=temperature,
                                                 initial_temperature="hi",
                                                 sweeps=10000)
        ising.metropolis()
        
        equilibrium = 5000
        equilibrium_energy = ising.energy_history[equilibrium:] / ising.no_of_sites
        equilibrium_magnetization = ising.magnetization_history[5000:] / ising.no_of_sites
        
        (energy_correlation_time, 
         energy, 
         energy_error) = analysis.binning_method(equilibrium_energy, 10, "Energy")
        (magnetization_correlation_time,
         magnetization,
         magnetization_error) = analysis.binning_method(np.absolute(equilibrium_magnetization), 10, "Magnetization")
        
        c = ising.heat_capacity(equilibrium_energy, temperature)
        specific_heat_trange.append((temperature, c))
        specific_heat_error_trange.append((temperature, analysis.bootstrap_method(equilibrium_energy, 100, temperature, ising.heat_capacity)))
        
        energy_trange.append((temperature, energy))
        energy_error_trange.append((temperature, energy_error))
        energy_correlation_trange.append((temperature, energy_correlation_time))
        
        magnetization_trange.append((temperature, np.absolute(magnetization)))
        magnetization_error_trange.append((temperature, magnetization_error))
        magnetization_correlation_trange.append((temperature, magnetization_correlation_time))
    
    return (energy_trange, 
            energy_error_trange,
            energy_correlation_trange, 
            magnetization_trange,
            magnetization_error_trange,
            magnetization_correlation_trange, 
            specific_heat_trange, 
            specific_heat_error_trange)

In [ ]:
def lattice_size_range(lattice_sizes, lower, upper, operation, step=0.2):
    lattice_sizes.sort()
    energy_data = []
    energy_error_data = []
    energy_correlation_data = []
    
    magnetization_data = []
    magnetization_error_data = []
    magnetization_correlation_data = []
    
    specific_heat_data = []
    specific_heat_error_data = []
    for p in lattice_sizes:
        print("Lattice Size is {0}.".format(p))
        (energy_trange, 
         energy_error_trange, 
         energy_correlation_trange, 
         magnetization_trange, 
         magnetization_error_trange, 
         magnetization_correlation_trange, 
         specific_heat_trange, 
         specific_heat_error_trange) = operation(p, lower, upper, step=step)
        
        energy_data.append(energy_trange)
        energy_error_data.append(energy_error_trange)
        energy_correlation_data.append(energy_correlation_trange)

        magnetization_data.append(magnetization_trange)
        magnetization_error_data.append(magnetization_error_trange)
        magnetization_correlation_data.append(magnetization_correlation_trange)

        specific_heat_data.append(specific_heat_trange)
        specific_heat_error_data.append(specific_heat_error_trange)
    
    #TODO: Add other correlation functions and different temperature range functions.
    for k in range(len(lattice_sizes)):
        show = False
        save=False
        if k == len(lattice_sizes) - 1:
            show = True
            save=True
        plotting.plot_correlation_time_range(magnetization_correlation_data[k], lattice_sizes[k], "Magnetization", show_plot=show, save=save)
    for k in range(len(lattice_sizes)):
        show = False
        save = False
        if k == len(lattice_sizes) - 1:
            show = True
        plotting.plot_correlation_time_range(energy_correlation_data[k], lattice_sizes[k], "Energy", show_plot=show, save=save)
    for k in range(len(lattice_sizes)):
        show = False
        exact_energy = None
        save = False
        if k == len(lattice_sizes) - 1:
            show = True
            save = True
            exact_energy = exact.internal_energy(1, lower, upper)
        plotting.plot_quantity_range(energy_data[k], energy_error_data[k], lattice_sizes[k], "Internal Energy per Spin", show_plot=show, exact=exact_energy, save=save)
    for k in range(len(lattice_sizes)):
        show = False
        exact_magnetization = None
        save = False
        if k == len(lattice_sizes) - 1:
            show = True
            save = True
            exact_magnetization = exact.magnetization(1, lower, upper)
        plotting.plot_quantity_range(magnetization_data[k], magnetization_error_data[k], lattice_sizes[k], "Magnetization per Spin", show_plot=show, exact=exact_magnetization, save=save)
    for k in range(len(lattice_sizes)):
        show = False
        exact_specific_heat = None
        save = False
        if k == len(lattice_sizes) - 1:
            show = True
            exact_specific_heat = exact.heat_capacity(1, lower, upper)
        plotting.plot_quantity_range(specific_heat_data[k], specific_heat_error_data[k], lattice_sizes[k], "Specific Heat per Spin", show_plot=show, exact=exact_specific_heat, save=save)

In [ ]:
lattice_size_range([4, 7, 10], 0.2, 5, binning_temperature_range)

In [ ]:
_, _, ct, data = analysis.binning_method(equilibrium_energy, 15, "Energy", False)
print(data, "\n")
print(analysis.jackknife(data[:38], 2, analysis.heat_capacity, temperature=2.2, no_of_sites=16), "\n")
print(analysis.jackknife(equilibrium_energy, 1000, analysis.heat_capacity, temperature=2.2, no_of_sites=16), "\n")

print(analysis.bootstrap_method(equilibrium_energy, 1000, analysis.heat_capacity, temperature=2.2, no_of_sites=16))

In [ ]:
print(analysis.jackknife(equilibrium_energy, 1000, analysis.heat_capacity, temperature=2.2, no_of_sites=16))
print(analysis.bootstrap_method(equilibrium_energy, 1000, analysis.heat_capacity, temperature=2.2, no_of_sites=16))

In [3]:
ising = ising.IsingModel(lattice_size=4, bond_energy=1, temperature=2.2, initial_temperature="hi", sweeps=10000)
plotting.show_lattice(ising.lattice)
ising.wolff(True)

AttributeError: 'IsingModel' object has no attribute 'IsingModel'